In [0]:

spark.sql('USE CATALOG dev_edh')
spark.sql('USE SCHEMA dummy')

In [0]:
%sql
--select * from dev_edh.dummy.lab_data_count_gold order by test_date desc

In [0]:
%python
# Lab_name = dbutils.jobs.taskValues.get(
#     "range_count", "Lab_name",
#     debugValue="default_lab_name"
# );print(Lab_name)

# lab = dbutils.jobs.taskValues.get(taskKey = "get_Lab_Name", key = "Lab_name", debugValue = "Lab A")

# print(f"Received value: {lab}")

In [0]:
from pyspark.sql import functions as F

# Static mapping for this example
lab_schedule = spark.createDataFrame([
    ("Lab A", "weekly"),
    ("Lab B", "monthly"),
    ("Lab D", "weekly")
], ["lab_name", "frequency"])

lab_schedule.show()


In [0]:
crnt_load_look_back = {"weekly": 7, "monthly": 30, "daily": 1}

In [0]:
%sql
--Date from the tables max
-- Select max(test_date) as Date from dev_edh.dummy.lab_data_count_gold

In [0]:
%sql
--CREATE WIDGET DROPDOWN Lab DEFAULT "Lab A" CHOICES SELECT * FROM (VALUES ("Lab A"), ("Lab B"), ("Lab D"));
--CREATE WIDGET DROPDOWN Date DEFAULT '2025-01-01' CHOICES select explode(sequence(DATE'2024-03-23',DATE'2025-12-31',INTERVAL 1 DAY )) ORDER BY col DESC 

In [0]:
#print(dbutils.widgets.get("Date"))

In [0]:
# getting the lab's name and Date
lab = spark.sql('select distinct test_source from dev_edh.dummy.lab_data_count_gold').collect()[0][0];
key_date = spark.sql('Select max(test_date) as Date from dev_edh.dummy.lab_data_count_gold').collect()[0][0];

display(key_date,lab)

In [0]:
#lab = dbutils.widgets.get("Lab")
frequency = lab_schedule.filter(F.col("lab_name") == lab).select("frequency").collect()[0][0]
crnt_look_back_day = crnt_load_look_back[frequency]
print(lab,frequency)
print(crnt_look_back_day)
#print(dbutils.widgets.get("Date"))

In [0]:
%python
import pandas as pd
from datetime import datetime

# change bucket
if frequency == "weekly":
    bucket = 42
elif frequency == "daily":
    bucket = 30
elif frequency == "monthly":
    bucket = 180
# Date value from the previous SQL tab to here as python value

#key_date = dbutils.widgets.get("Date")
#lab = dbutils.widgets.get("Lab")
look_back_days = bucket+1

# run_date = pd.to_datetime(
#     pd.to_datetime(key_date) - pd.DateOffset(days=bucket), format="%Y-%m-%d"
# )   # We can change it in poroduction by taking the date from system

# Add 1 day on the run day
run_date = pd.to_datetime(pd.to_datetime(key_date) + pd.DateOffset(days=1), format="%Y-%m-%d")
print(key_date, look_back_days)
print('run_date-',run_date)
current_end_dt = pd.to_datetime(key_date) 
    # current_end_dt = pd.to_datetime(pd.to_datetime(key_date) - pd.DateOffset(days=1), format="%Y-%m-%d") 
print('current_end_dt -',current_end_dt)

current_start_dt = pd.to_datetime(
    pd.to_datetime(key_date) - pd.DateOffset(days=crnt_look_back_day ), format="%Y-%m-%d"
)
print('current_start_dt -',current_start_dt)
lookback_end_dt = pd.to_datetime(
    pd.to_datetime(current_start_dt) - pd.DateOffset(days=1), format="%Y-%m-%d"
)
lookback_start_dt = pd.to_datetime(
    pd.to_datetime(current_start_dt) - pd.DateOffset(days=look_back_days),
    format="%Y-%m-%d",
)

spark.conf.set("key.date", key_date)
key_date = datetime.strptime(key_date, '%Y-%m-%d').strftime('%Y-%m-%d')
spark.conf.set("key.date", key_date)
#display(key_date)

spark.conf.set("look.back.days", look_back_days)
# # assign values for pyspark
current_end_dt = str(current_end_dt)
current_start_dt = str(current_start_dt)
lookback_end_dt = str(lookback_end_dt)
lookback_start_dt = str(lookback_start_dt)
bucket = str(bucket)
spark.conf.set("current.end.date", current_end_dt)
spark.conf.set("current.start.date", current_start_dt)
spark.conf.set("lookback.end.date", lookback_end_dt)
spark.conf.set("lookback.start.date", lookback_start_dt)
spark.conf.set("week.days", crnt_look_back_day)
spark.conf.set("test.source.name", lab)
spark.conf.set("lab.bucket", bucket)

In [0]:
%sql
--Change format and print just to check
CREATE OR REPLACE TEMPORARY VIEW temp_view AS
select
'${test.source.name}' as lab_name,
'${key.date}' as key_date_entered,
  '${look.back.days}' as number_of_days_back,
  cast('${current.end.date}' as date) as current_end_dt,
  cast('${current.start.date}' as date) as current_start_dt,
  cast('${lookback.end.date}' as date) as lookback_end_dt,
  cast('${lookback.start.date}' as date) as lookback_start_dt,
  '${week.days}' as crnt_look_back_day;

select * FROM temp_view
  --select key_date_entered from temp_view

In [0]:
%sql
-- Current load count
create or replace temp view current_data_load as

select distinct test_source,cast('${current.start.date}' as date) as current_st_dt, cast('${current.end.date}' as date) as current_end_dt,sum(total_load) as total_load
from dev_edh.dummy.lab_data_count_gold
where test_date between '${current.start.date}' and '${current.end.date}' 
and test_source = '${test.source.name}'
group by test_source;

select * from current_data_load 


In [0]:
%sql
-- Lookback load count
create or replace temp view lookback_data_load as

select test_source,
  cast(date_trunc('month', test_date) as date) as month,
  sum(total_load) as monthly_load
from dev_edh.dummy.lab_data_count_gold
where test_date between '${lookback.start.date}' and '${lookback.end.date}' 
and test_source = '${test.source.name}'
group by test_source, date_trunc('month', test_date);

-- Calculate Range start and Range end value from lookback data
create or replace temp view range_start_end as
select round(avg(monthly_load) - stddev(monthly_load)) as range_start,
       round(avg(monthly_load) + stddev(monthly_load)) as range_end,
       min(month) as lookback_start_date, max(month) as lookback_end_date, '${test.source.name}'as test_source
from lookback_data_load;

select * from range_start_end

In [0]:
%sql
-- Join the 2 temp view 'range_start_end' and 'current_data_load'
create or replace temp view current_data_status as
select t1.test_source,t1.lookback_start_date,t1.lookback_end_date,t2.current_st_dt,t2.current_end_dt,t1.range_start,t1.range_end,t2.total_load,
case when t2.total_load between t1.range_start and t1.range_end then 'within range' else 'out of range' end as status
from range_start_end t1
join current_data_load t2 
on t1.test_source = t2.test_source;

select * from current_data_status
    


In [0]:
# %sql
# -- calculate load for each month dynamically based on the frequency  [this is working as expected , but I changed this to temp view]
# with TEMP as(
# select distinct * from(
# select 
#   test_source,
#   cast(date_trunc('month', test_date) as date) as dates,
#   sum(total_load) over (partition by test_source, date_trunc('month', test_date)) as monthly_load
# from dev_edh.dummy.lab_data_count_gold
# where test_source = '${test.source.name}' and test_date between '${lookback.start.date}' and '${lookback.end.date}'
# group by test_source, date_trunc('month', test_date), test_date,total_load)
# )
# select round(avg(monthly_load) - stddev(monthly_load))  as range_start,round(avg(monthly_load) + stddev(monthly_load)) as range_end from TEMP
# group by test_source

In [0]:
%sql
create table if not exists dev_edh.dummy.lab_data_status (
  test_source string,
  range_start double,
  range_end double,
  current_load bigint,
  load_start_date date,
  load_end_date date,
  current_start_date date,
  current_end_date date,
  status string,
  timestamp timestamp
);

-- insert into dev_edh.dummy.lab_data_status
-- select 
--   current_load.test_source,
--   range_start,
--   range_end,
--   total_load as current_load,
--   cast('${lookback.start.date}' as date) as load_start_date,
--   cast('${lookback.end.date}' as date) as load_end_date,
--   cast('${current.start.date}' as date) as current_start_date,
--   cast('${current.end.date}' as date) as current_end_date,
--   case 
--     when total_load between range_start and range_end then 'within range' 
--     else 'out of range' 
--   end as status,
--   current_timestamp() as timestamp      
-- from (
--   select 
--     test_source,
--     count(*) as total_load
--   from dev_edh.dummy.lab_data_count_gold
--   where test_source = '${test.source.name}' and test_date between '${current.start.date}' and '${current.end.date}'
--   group by test_source
-- ) as current_load
-- cross join (
--   select 
--     test_source,
--     round(avg(monthly_load) - stddev(monthly_load)) as range_start,
--     round(avg(monthly_load) + stddev(monthly_load)) as range_end
--   from (
--     select 
--       test_source,
--       cast(date_trunc('month', test_date) as date) as dates,
--       count(*) over (partition by test_source, date_trunc('month', test_date)) as monthly_load
--     from dev_edh.dummy.lab_data_count_gold
--     where test_source = '${test.source.name}' and test_date between '${lookback.start.date}' and '${lookback.end.date}'
--     group by test_source, date_trunc('month', test_date), test_date
--   ) as TEMP
--   group by test_source
-- ) as range_values;

In [0]:
%sql
--create a merge statement and update only into the table 'lab_data_status' if and only if the test_source, lookback_start_date, lookback_end_date, current_start_date, current_end_date, are different otherwise insert a new row using the temp view current_data_status
MERGE INTO dev_edh.dummy.lab_data_status AS target
USING current_data_status AS source
ON target.test_source = source.test_source 
AND target.load_start_date = source.lookback_start_date 
AND target.load_end_date = source.lookback_end_date 
AND target.current_start_date = source.current_st_dt 
AND target.current_end_date = source.current_end_dt
WHEN MATCHED THEN
  UPDATE SET
    target.range_start = source.range_start,
    target.range_end = source.range_end,
    target.current_load = source.total_load,
    target.status = source.status,
    target.timestamp = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT (test_source, range_start, range_end, current_load, load_start_date, load_end_date, current_start_date, current_end_date, status, timestamp)
  VALUES (source.test_source, source.range_start, source.range_end, source.total_load, source.lookback_start_date, source.lookback_end_date, source.current_st_dt, source.current_end_dt, source.status, current_timestamp())

In [0]:
# %sql
# MERGE INTO dev_edh.dummy.lab_data_status AS target
# USING (
#   SELECT 
#     current_load.test_source,
#     range_start,
#     range_end,
#     total_load AS current_load,
#     CAST('${lookback_start_dt}' AS DATE) AS load_start_date,
#     CAST('${lookback_end_dt}' AS DATE) AS load_end_date,
#     CAST('${current_start_dt}' AS DATE) AS current_start_date,
#     CAST('${current_end_dt}' AS DATE) AS current_end_date,
#     CASE 
#       WHEN total_load BETWEEN range_start AND range_end THEN 'within range' 
#       ELSE 'out of range' 
#     END AS status,
#     current_timestamp() AS timestamp      
#   FROM (
#     SELECT 
#       test_source,
#       COUNT(*) AS total_load
#     FROM dev_edh.dummy.lab_data_count_gold
#     WHERE test_source = '${lab}' AND test_date BETWEEN '${current_start_dt}' AND '${current_end_dt}'
#     GROUP BY test_source
#   ) AS current_load
#   CROSS JOIN (
#     SELECT 
#       test_source,
#       ROUND(AVG(monthly_load) - STDDEV(monthly_load)) AS range_start,
#       ROUND(AVG(monthly_load) + STDDEV(monthly_load)) AS range_end
#     FROM (
#       SELECT 
#         test_source,
#         CAST(DATE_TRUNC('month', test_date) AS DATE) AS dates,
#         COUNT(*) OVER (PARTITION BY test_source, DATE_TRUNC('month', test_date)) AS monthly_load
#       FROM dev_edh.dummy.lab_data_count_gold
#       WHERE test_source = '${lab}' AND test_date BETWEEN '${lookback_start_dt}' AND '${lookback_end_dt}'
#       GROUP BY test_source, DATE_TRUNC('month', test_date), test_date
#     ) AS TEMP
#     GROUP BY test_source
#   ) AS range_values
# ) AS source
# ON target.test_source = source.test_source
#   AND target.load_start_date = source.load_start_date
#   AND target.load_end_date = source.load_end_date
#   AND target.current_start_date = source.current_start_date
#   AND target.current_end_date = source.current_end_date
# WHEN MATCHED THEN
#   UPDATE SET
#     target.range_start = source.range_start,
#     target.range_end = source.range_end,
#     target.current_load = source.current_load,
#     target.status = source.status,
#     target.timestamp = source.timestamp
# WHEN NOT MATCHED THEN
#   INSERT (test_source, range_start, range_end, current_load, load_start_date, load_end_date, current_start_date, current_end_date, status, timestamp)
#   VALUES (source.test_source, source.range_start, source.range_end, source.current_load, source.load_start_date, source.load_end_date, source.current_start_date, source.current_end_date, source.status, source.timestamp)

In [0]:
# Update the table lab_data_status add a field called timestamp. Which will store the running timestamp
# spark.sql("""
# ALTER TABLE dev_edh.dummy.lab_data_status
# ADD COLUMNS (timestamp TIMESTAMP)
# """)


In [0]:
%sql
SELECT * FROM dev_edh.dummy.lab_data_status 

In [0]:
%python
# Remove widgets lab
#dbutils.widgets.remove("Date")